In [1]:
import pandas
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import colors
import cProfile
import random
from ripser import ripser
from persim import plot_diagrams
import os
from os.path import join, isfile, exists

In [ ]:
#for a given timepoint, computes the persistent homology of all samples
#    at that timepoint
#note that the function assumes the timepoint data files are stored
#    in folders according to timepoint
#also, the files contained the homology are stored in folders by timepoint
#-dataFolder, parent directory of timepoint folders
#-xName, column name of x-coordinate values in file
#-yName, column name of y-coordinate values in file
#-outFol0, name of parent directory of dimension 0 homology folders
#-outFol1, name of parent directory of dimension 1 homology folders
#-mkDgm, string that will be the filename of the graph of the homology
#    set to 0 if no such graph is desired
#-xName, column name of x-coordinate values in file
#-yName, column name of y-coordinate values in file
def createDiagram(tmpt,condFile,dataFolder,outFol0,outFol1,mkDgm = 0,xName='xval',yName = 'yval')
    InDict = {}
    folder_path = dataFolder+'/'+timepoint
    files = [f for f in os.listdir(folder_path) if isfile(join(folder_path, f))]
    for file in files:
        currKey = file[:-4]        
        try:
            file_path = os.path.join(folder_path, file)
            currDF = pandas.read_csv(file_path)
            currSamp = [currDF[xName].tolist(),currDF[yName].tolist()]
            forRipser = np.array([[currSamp[0][j],currSamp[1][j]] for j in range(len(currSamp[0]))])
            theDiagrams = ripser(forRipser)['dgms']
            dgms0 = [j for j in theDiagrams[0] if (j[0]!=np.Inf and j[1]!=np.Inf)]
            dgms1 = [j for j in theDiagrams[1] if (j[0]!=np.Inf and j[1]!=np.Inf)]
        
            if mkDgm != 0:
                limrangeReg =np.array([150,np.array([k[1] for k in dgms0]).max(),np.array([k[1] for k in dgms1]).max()]).max()
                figl = plt.figure()
                plot_diagrams(theDiagrams, xy_range = [-1,limrangeReg,-1,limrangeReg], show=True)
                figl.savefig(mkDgm+'.png',bbox_inches='tight')
        
            df0 = pandas.DataFrame(data = {'starts':[j[0] for j in dgms0], 'ends':[j[1] for j in dgms0]})
            df0.to_csv(outFol0+'/'+tmpt+'/'+currKey+'.csv', index = False) 
            df1 = pandas.DataFrame(data = {'starts':[j[0] for j in dgms1], 'ends':[j[1] for j in dgms1]})
            df1.to_csv(outFol1+'/'+tmpt+'/'+currKey+'.csv', index = False)
        except:
            pass